# **Practical assignment for Topic 11**

In the assignment, you have to use the dataset `auto-lpkm.tsv` given in the lecture and you should use the **gplearn** library.

Your assignment consists of **just one task**:
<br>
<br>

**Task 1**<br>
Explore configurations of the algorithm and try to find a better configuration than the one used in the lecture. Approach this task systematically. How results depend on the parameters?

Special notes to follow:
* You are not allowed to set `population_size` larger than 200.
* You are not allowed to set `generations` larger than 20.
* You have to keep `max_samples=1`, `random_state=42`, and `warm_start=False`.

When you are finished, write conclusions about the process and the findings.

---
**After the tasks are done, submit this file. Do not clear it's output - all print-outs and diagrams should be left in the file.**